In [1]:
!pip install fastparquet -q


[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import gc
import os
import pickle
import random
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

In [3]:
class CFG:
    predict = True
    PREPROCESS = False
    EPOCHS = 18
    BATCH_SIZE = 4096
    LR = 1e-3
    WD = 0.05

    NBR_FOLDS = 15
    SELECTED_FOLDS = [0]

    SEED = 2024

In [4]:
import tensorflow as tf
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

set_seeds(seed=CFG.SEED)

D0625 13:07:13.474424543      14 config.cc:119]                        gRPC EXPERIMENT tcp_frame_size_tuning               OFF (default:OFF)
D0625 13:07:13.474447810      14 config.cc:119]                        gRPC EXPERIMENT tcp_rcv_lowat                       OFF (default:OFF)
D0625 13:07:13.474451732      14 config.cc:119]                        gRPC EXPERIMENT peer_state_based_framing            OFF (default:OFF)
D0625 13:07:13.474454490      14 config.cc:119]                        gRPC EXPERIMENT flow_control_fixes                  ON  (default:ON)
D0625 13:07:13.474457074      14 config.cc:119]                        gRPC EXPERIMENT memory_pressure_controller          OFF (default:OFF)
D0625 13:07:13.474459943      14 config.cc:119]                        gRPC EXPERIMENT unconstrained_max_quota_buffer_size OFF (default:OFF)
D0625 13:07:13.474462498      14 config.cc:119]                        gRPC EXPERIMENT new_hpack_huffman_decoder           ON  (default:ON)
D0625 13:07:13.

In [5]:
import tensorflow as tf

# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect(tpu="local") # "local" for 1VM TPU
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
    print("REPLICAS: ", strategy.num_replicas_in_sync)
    all_data = True
except tf.errors.NotFoundError:
    all_data = False

    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print("Not on TPU")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU

# Preprocessing

In [6]:
!pip install tokenizers
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.6/402.6 kB 24.2 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.2/776.2 kB 34.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.1
[notice] To update, run: pip install --upgrade pip


# Create a tokenizer

We use Tokenizer and PreTrainedTokenizerFast,

First create the dictionary enc.

In [7]:
from tokenizers import Tokenizer
from transformers import PreTrainedTokenizerFast
from tokenizers.models import WordLevel,BPE

from tokenizers.pre_tokenizers import Whitespace,Split,ByteLevel

from tokenizers.normalizers import Lowercase, NFKC

enc = { '[PAD]':0,
        'Br':1, 'C':2, 'N':3, 'O':4, 'H':5, 'S':6, 'F':7, 'Cl':8, 'B':9, 'I':10, 
        's':11,'o':12, 'c':13, 'n':14, 'i':15, # is Atomic
        '.':16 ,'=':17 ,'#':18, # bond
        '/':19, # direction
        '-':20, '+': 21, # charge
        '[':22,']':23, # Atomic mass
        '(':24,')':25, # Branches
        '@@':26, '@':27, # tetrahedron
        '1':28,'2':29,'3':30,'4':31,'5':32,'6':33,'7':34,'8':35,'9':36
      }

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Then create a Tokenizer with BPE
Pay attention to ```merges```. In the case that both @ and @@ exist, we should find @@ first, then @

then converted to PreTrainedTokenizerFast

Create special tokens at the same time

Also, we should set [Dy] to sep_token

In [8]:
tokenizer = Tokenizer(BPE(vocab=enc, unk_token="[UNK]",merges=[('@','@')]))

tokenizer_fast = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
tokenizer_fast.add_special_tokens({'pad_token': '[PAD]','sep_token':'[Dy]'})
tokenizer_fast.add_special_tokens({'additional_special_tokens':['Br','Cl']})

len(set(enc.values()))

37

In [9]:
tokenizer_fast.all_special_ids

[37, 0, 1, 8]

# Then implement batch spliting.

In [10]:
# def compile_model():
#     with strategy.scope():
#         model = tf.keras.Sequential([
#             tf.keras.layers.Dense(64, input_shape=(64,))
#         ])
#         model.compile(metrics=['accuracy'])
#         return model
# model = compile_model()
# dummy_input = tf.random.uniform((1, 64))
# output = model(dummy_input)
# print("TPU shape:", output.shape)


In [11]:
from joblib import Parallel, delayed

def encode_smiles_batch(smiles_chunk, max_length=142, batch_size=32):
    encoded_batch = tokenizer_fast.batch_encode_plus(
        smiles_chunk,
        max_length=max_length,
        padding='max_length',
        return_tensors='np',
        truncation=True
    )['input_ids']

    return encoded_batch.astype(np.int8)

def encode_smiles_parallel(smiles, max_length=142, batch_size=4096*8):
    smiles_chunks = [smiles[i:i + batch_size] for i in range(0, len(smiles), batch_size)]
    encoded_chunks = Parallel(n_jobs=-1)(delayed(encode_smiles_batch)(chunk, max_length, batch_size) for chunk in tqdm(smiles_chunks))
    
    encoded_smiles = np.concatenate(encoded_chunks, axis=0)
    
    return encoded_smiles


# TEST

In [12]:
tokenizer_fast.batch_encode_plus(['BrBC#CCOc1ccc(CNc2nc(Nc3cc(-c4ccc(Cl)cc4)sc3C(=O)OC)nc(N[C@@H](CC#C)CC(=O-)N[Dy])n2)cc1'],max_length=142,padding='max_length')


{'input_ids': [[1, 9, 2, 18, 2, 2, 4, 13, 28, 13, 13, 13, 24, 2, 3, 13, 29, 14, 13, 24, 3, 13, 30, 13, 13, 24, 20, 13, 31, 13, 13, 13, 24, 8, 25, 13, 13, 31, 25, 11, 13, 30, 2, 24, 17, 4, 25, 4, 2, 25, 14, 13, 24, 3, 22, 2, 26, 5, 23, 24, 2, 2, 18, 2, 25, 2, 2, 24, 17, 4, 20, 25, 3, 37, 25, 14, 29, 25, 13, 13, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## We can see that Br, Cl and @@ have been successfully separated

In [13]:
tokenizer_fast.tokenize('BrBC#CCOc1ccc(CNc2nc(Nc3cc(-c4ccc(Cl)cc4)sc3C(=O)OC)nc(N[C@@H](CC#C)CC(=O-)N[Dy])n2)cc1')


['Br',
 'B',
 'C',
 '#',
 'C',
 'C',
 'O',
 'c',
 '1',
 'c',
 'c',
 'c',
 '(',
 'C',
 'N',
 'c',
 '2',
 'n',
 'c',
 '(',
 'N',
 'c',
 '3',
 'c',
 'c',
 '(',
 '-',
 'c',
 '4',
 'c',
 'c',
 'c',
 '(',
 'Cl',
 ')',
 'c',
 'c',
 '4',
 ')',
 's',
 'c',
 '3',
 'C',
 '(',
 '=',
 'O',
 ')',
 'O',
 'C',
 ')',
 'n',
 'c',
 '(',
 'N',
 '[',
 'C',
 '@@',
 'H',
 ']',
 '(',
 'C',
 'C',
 '#',
 'C',
 ')',
 'C',
 'C',
 '(',
 '=',
 'O',
 '-',
 ')',
 'N',
 '[Dy]',
 ')',
 'n',
 '2',
 ')',
 'c',
 'c',
 '1']

## TEST on testset

only 5 minutes

In [14]:
# test_raw = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
# smiles = test_raw['molecule_smiles'].values

# smiles_enc = encode_smiles_batch(smiles.tolist(),max_length=142,batch_size = 4096*4)

# smiles_enc = np.stack(smiles_enc)

# test = pd.DataFrame(smiles_enc, columns = [f'm_enc{i}' for i in range(142)])
# test.to_parquet('test_enc.parquet')
# 100%|██████████| 103/103 [03:49<00:00,  2.23s/it]

# Preprocess

Since some libraries, such as tokenizers, initialize some global states when used,  these states can cause problems after the process forks. So we'll explicitly set the environment variable ```TOKENIZERS_PARALLELISM=false``` to disable parallel processing for tokenizers. This will ensure that parallel processing is  no longer used after the process forks.

In [15]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [16]:
if CFG.PREPROCESS:
    if all_data:
        train_raw = pd.read_parquet('/kaggle/input/leash-BELKA/train.parquet')
        smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values
        assert (smiles!=train_raw[train_raw['protein_name']=='HSA']['molecule_smiles'].values).sum() == 0
        assert (smiles!=train_raw[train_raw['protein_name']=='sEH']['molecule_smiles'].values).sum() == 0

    else:
        train_raw = pd.read_csv('/kaggle/input/leash-BELKA/train.csv',nrows = 1000)
        smiles = train_raw[train_raw['protein_name']=='BRD4']['molecule_smiles'].values[:333]
        HSA = train_raw[train_raw['protein_name']=='HSA'].reset_index(drop=True)
        sEH = train_raw[train_raw['protein_name']=='sEH'].reset_index(drop=True)
        assert (smiles!=HSA['molecule_smiles'].values).sum() == 0
        assert (smiles!=sEH['molecule_smiles'].values).sum() == 0

    smiles_enc = encode_smiles_parallel(smiles.tolist(), max_length=142, batch_size=4096*8)
    del smiles
    gc.collect()
    smiles_enc = np.stack(smiles_enc)

    
    train = pd.DataFrame(smiles_enc, columns = [f'm_enc{i}' for i in range(142)])
    
    train['bind1'] = train_raw[train_raw['protein_name']=='BRD4']['binds'].values
    train['bind2'] = train_raw[train_raw['protein_name']=='HSA']['binds'].values
    train['bind3'] = train_raw[train_raw['protein_name']=='sEH']['binds'].values
    train.to_parquet('train_enc.parquet')

    test_raw = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
    smiles = test_raw['molecule_smiles'].values

    smiles_enc = encode_smiles_batch(smiles.tolist(),max_length=142,batch_size = 4096*8)
    
    smiles_enc = np.stack(smiles_enc)
    
    test = pd.DataFrame(smiles_enc, columns = [f'm_enc{i}' for i in range(142)])
    test.to_parquet('test_enc.parquet')

else:
    if not CFG.predict:
        train = pd.read_parquet('/kaggle/input/belka-chemical-encoding/train_enc.parquet')
    test = pd.read_parquet('/kaggle/input/belka-chemical-encoding/test_enc.parquet')

In [17]:
print('done')

done


# Modeling

In [18]:
import tensorflow as tf
def my_model():
    with strategy.scope():
        INP_LEN = 142
        NUM_FILTERS = 32
        hidden_dim = 128
        tokens_num = 37

        inputs = tf.keras.layers.Input(shape=(INP_LEN,), dtype='int32')
        x = tf.keras.layers.Embedding(input_dim=tokens_num, output_dim=hidden_dim, input_length=INP_LEN, mask_zero = True)(inputs)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*2, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.Conv1D(filters=NUM_FILTERS*3, kernel_size=3,  activation='relu', padding='valid',  strides=1)(x)
        x = tf.keras.layers.GlobalMaxPooling1D()(x)

        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(0.1)(x)

        outputs = tf.keras.layers.Dense(3, activation='sigmoid')(x)

        model = tf.keras.models.Model(inputs = inputs, outputs = outputs)
        optimizer = tf.keras.optimizers.AdamW(learning_rate=CFG.LR, weight_decay=CFG.WD)
        loss = 'binary_crossentropy'
        weighted_metrics = [tf.keras.metrics.AUC(curve='PR', name = 'avg_precision')]
        model.compile(
        loss=loss,
        optimizer=optimizer,
        weighted_metrics=weighted_metrics,
        )
        return model
model = my_model()

model.summary()
tf.keras.backend.clear_session()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 142)]             0         
                                                                 
 embedding (Embedding)       (None, 142, 128)          4736      
                                                                 
 conv1d (Conv1D)             (None, 140, 32)           12320     
                                                                 
 conv1d_1 (Conv1D)           (None, 138, 64)           6208      
                                                                 
 conv1d_2 (Conv1D)           (None, 136, 96)           18528     
                                                                 
 global_max_pooling1d (Globa  (None, 96)               0         
 lMaxPooling1D)                                                  
                                                             

# Train & Inference

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score as APS
FEATURES = [f'm_enc{i}' for i in range(142)]
TARGETS = ['bind1', 'bind2', 'bind3']
skf = StratifiedKFold(n_splits = CFG.NBR_FOLDS, shuffle = True, random_state = 42)

all_preds = []
if not CFG.predict:
    for fold,(train_idx, valid_idx) in enumerate(skf.split(train, train[TARGETS].sum(1))):

        if fold not in CFG.SELECTED_FOLDS:
            continue;

        X_train = train.loc[train_idx, FEATURES]
        y_train = train.loc[train_idx, TARGETS]
        X_val = train.loc[valid_idx, FEATURES]
        y_val = train.loc[valid_idx, TARGETS]

        es = tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_loss", mode='min', verbose=1)
        checkpoint = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', filepath=f"model-{fold}.tf",
                                                            save_best_only=True, save_weights_only=True,
                                                        mode='min')
        reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.05, patience=4, verbose=1)
        model = my_model()
        history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=CFG.EPOCHS,
                callbacks=[checkpoint, reduce_lr_loss, es],
                batch_size=CFG.BATCH_SIZE,
                verbose=1,
            )
        model.load_weights(f"model-{fold}.tf")
        model.load_weights(f"/kaggle/input/re-encode-model/model-{fold}.tf")

        oof = model.predict(X_val, batch_size = CFG.BATCH_SIZE)
        print('fold :', fold, 'CV score =', APS(y_val, oof, average = 'micro'))

        preds = model.predict(test, batch_size = CFG.BATCH_SIZE)
        all_preds.append(preds)
if CFG.predict:
    model.load_weights(f"/kaggle/input/re-encode-model/model-0.tf")
    preds = model.predict(test, batch_size = CFG.BATCH_SIZE)
    all_preds.append(preds)
preds = np.mean(all_preds, 0)

2024-06-25 13:08:14.698471: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2024-06-25 13:08:14.765407: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


409/409 [==============================] - 8s 12ms/step


# Submission

In [20]:
tst = pd.read_parquet('/kaggle/input/leash-BELKA/test.parquet')
tst['binds'] = 0
tst.loc[tst['protein_name']=='BRD4', 'binds'] = preds[(tst['protein_name']=='BRD4').values, 0]
tst.loc[tst['protein_name']=='HSA', 'binds'] = preds[(tst['protein_name']=='HSA').values, 1]
tst.loc[tst['protein_name']=='sEH', 'binds'] = preds[(tst['protein_name']=='sEH').values, 2]
tst[['id', 'binds']].to_csv('submission.csv', index = False)